In [1]:
import os
import re
import anndata as ad
import numpy as np
import pandas as pd
import scanpy as sc
import scipy as sci
import seaborn as sns; sns.set(color_codes=True)
import matplotlib.pyplot as plt
%matplotlib inline
import scimap as sm

import tqdm as tqdm
from joblib import Parallel, delayed

/home/smithd30/miniconda3/envs/scimap/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [2]:
root_dir = "../../outs/neighborhood_analysis"
dat_dir = [d for d in os.listdir(root_dir) if 'reg' in d]


In [50]:
def manual_gating(adata, gate_df):
    # STROMAL, FDC
    # reset the label
    adata.obs.loc[adata.obs['cell_type_pred_knn'] == 'FDC', "new_pheno"] = "Stromal"
    m = "CD45"
    thresh = gate_df.loc[m,:].values[0]
    cond1 = adata.X[:,adata.var_names == m] < thresh
    cond1 = cond1.flatten()
    cond2 = adata.obs['cell_type_pred_knn'] == 'FDC'
    adata.obs.loc[(cond1) & (cond2), 'new_pheno'] = "FDC"
    
    # STROMAL
    m = "CD45"
    thresh = gate_df.loc[m,:].values[0]
    cond1 = adata.X[:,adata.var_names == m] < thresh
    cond1 = cond1.flatten()
    cond2 = adata.obs['cell_type_pred_knn'] == 'Endothelial'
    adata.obs.loc[(cond1) & (cond2), 'new_pheno'] = "Stromal"
    
    # ENDOTHELIAL
    m1 = "CD31"
    thresh = gate_df.loc[m1,:].values[0]
    cond1 = adata.X[:,adata.var_names == m1] > thresh
    cond1 = cond1.flatten()
    cond2 = adata.obs['cell_type_pred_knn'] == 'Endothelial'
    m2 = "CD34"
    thresh = gate_df.loc[m2,:].values[0]
    cond3 = adata.X[:,adata.var_names == m2] > thresh
    cond3 = cond1.flatten()
    adata.obs.loc[(cond1) & (cond2) & (cond3), 'new_pheno'] = "Endothelial"

    # B-CELL AND B-CELL GERMINAL
    adata.obs.loc[adata.obs['cell_type_pred_knn'] == 'B_Cell_germ', "new_pheno"] = "B_Cell"
    m1 = "BCL-2"
    thresh = gate_df.loc[m1,:].values[0]
    cond1 = adata.X[:,adata.var_names == m1] > thresh
    cond1 = cond1.flatten()
    cond2 = adata.obs['new_pheno'].isin(['cDC1', 'Stromal', 'unknown', 'FDC']) #added FDC, 12/19
    m2 = "CD20"
    thresh = gate_df.loc[m2,:].values[0]
    cond3 = adata.X[:,adata.var_names == m2] > thresh
    cond3 = cond3.flatten()
    cond4 = adata.obs['new_pheno'] == 'B_Cell'
    # cond2 = cond2 | cond4
    #     B-cell
    adata.obs.loc[((cond2) | (cond4)) & (cond3 | cond1), 'new_pheno'] = "B_Cell"
    #     B-cell, germinal
    # cond1 = adata.X[:,adata.var_names == m1] < thresh
    # cond1 = cond1.flatten()
    # adata.obs.loc[(cond1) & ((cond2) | (cond4)) & (cond3), 'new_pheno'] = "B_Cell, germinal"
    
    # 12/16, FLIP FDC TO STROMAL, doing this after B_cell so we don't accidentally lose them
    adata.obs.loc[adata.obs['new_pheno']=='FDC', 'new_pheno'] = 'Stromal'
    
    # MACROPHAGE
    adata.obs.loc[adata.obs['cell_type_pred_knn'].isin(['Macrophage_M2','Sinus_histiocyte']), "new_pheno"] = "Macrophage"
    m1 = "CD68"
    thresh = gate_df.loc[m1,:].values[0]
    cond1 = adata.X[:,adata.var_names == m1] > thresh
    cond1 = cond1.flatten()
    cond2 = adata.obs['cell_type_pred_knn'] == 'Neutrophil'
    m2 = "CD163"
    thresh = gate_df.loc[m2,:].values[0]
    cond3 = adata.X[:,adata.var_names == m2] > thresh
    cond3 = cond3.flatten()
    adata.obs.loc[((cond1) | (cond3)) & (cond2), 'new_pheno'] = "Macrophage"
    
    # PLASMA CELL, PROLIFERATING
    adata.obs.loc[adata.obs['cell_type_pred_knn'] == 'Plasma_prolif', "new_pheno"] = "unknown"
    m1 = "CD138"
    thresh = gate_df.loc[m1,:].values[0]
    cond1 = adata.X[:,adata.var_names == m1] > thresh
    cond1 = cond1.flatten()
    cond2 = adata.obs['cell_type_pred_knn'] == 'Plasma_prolif'
    adata.obs.loc[(cond1) & (cond2), 'new_pheno'] = "Plasma_Cell"
    # m3 = "Ki67"
    # thresh = gate_df.loc[m3,:].values[0]
    # cond3 = adata.X[:,adata.var_names == m3] > thresh
    # cond3 = cond1.flatten()
    # cond4 = adata.obs['cell_type_pred_knn'] == 'Plasma_Cell'
    # adata.obs.loc[((cond2) | (cond4)) & (cond3), 'new_pheno'] = "Plasma_Cell, prolif."
    
    # cDC1
    adata.obs.loc[adata.obs['cell_type_pred_knn'] == 'cDC1', "new_pheno"] = "unknown"
    m1 = "CLEC9A"
    thresh = gate_df.loc[m1,:].values[0]
    cond1 = adata.X[:,adata.var_names == m1] > thresh
    cond1 = cond1.flatten()
    cond2 = adata.obs['new_pheno'].isin(['B_Cell','unknown', 'Stromal'])
    m2 = "IRF8"
    thresh = gate_df.loc[m2,:].values[0]
    cond3 = adata.X[:,adata.var_names == m2] > thresh
    cond3 = cond3.flatten()
    m3 = "CD1c"
    thresh = gate_df.loc[m3,:].values[0]
    cond4 = adata.X[:,adata.var_names == m3] > thresh
    cond4 = cond4.flatten()
    adata.obs.loc[(cond2) & ((cond3 & cond4) | (cond3 & cond1)), 'new_pheno'] = "cDC1"
    
    # T cells
    adata.obs.loc[adata.obs['cell_type_pred_knn'] == 'activated_T_Cell', "new_pheno"] = "unknown"
    
    return adata

In [5]:
def pheno_unknown(adata, gate_df, phenotype):
    adata_unknown = adata[adata.obs['new_pheno'] == "unknown",:]
    adata_unknown = sm.pp.rescale(adata_unknown, gate=gate_df, imageid='orig.ident')
    adata_unknown = sm.tl.phenotype_cells(adata_unknown, phenotype=phenotype, label="phenotype")
    adata.obs.loc[adata.obs['new_pheno']=="unknown", "new_pheno"] = adata_unknown.obs['phenotype']
    return adata

In [5]:
phenotype = pd.read_csv('unknown_pheno_matrix.csv')
for d in dat_dir:
    adata = sc.read_h5ad(os.path.join(root_dir,d,'dat.h5ad'))
    # processing the data
    adata.X = adata.raw.X
    adata.X = adata.X.toarray()
    adata.raw = adata
    adata.X = np.log1p(adata.X)
    # doing the manual corrections
    adata.obs['new_pheno'] = adata.obs['cell_type_pred_knn'].astype(str)
    gate_df = pd.read_csv('cohort1_gates_orig_v2.csv', header=None, index_col=0, names=['gate'])
    adata = manual_gating(adata, gate_df)
    # assigning the unknowns
    gate_df = pd.read_csv('cohort1_gates_orig_v2.csv', header=None)
    adata = pheno_unknown(adata, gate_df, phenotype)
    
    # accept likely-cDC1
    adata.obs.loc[adata.obs['new_pheno'] == 'likely-cDC1', 'new_pheno'] = 'cDC1'
    adata.obs.to_csv(os.path.join('../forQuPath/Data/',d,'annotation_df_v2.csv'))
    adata.write(os.path.join(root_dir,d,'gated_dat.h5ad'))

GMM for ['SeuratProject']
Categories (1, object): ['SeuratProject']
Finding the optimal gate by GMM for DAPI
Finding the optimal gate by GMM for IRF4
Finding the optimal gate by GMM for BCL-6
Finding the optimal gate by GMM for FOXP3
Finding the optimal gate by GMM for CD25
Finding the optimal gate by GMM for CD40
Finding the optimal gate by GMM for CD10
Finding the optimal gate by GMM for CD134
Finding the optimal gate by GMM for CD69
Finding the optimal gate by GMM for ICOS
Finding the optimal gate by GMM for CD30
Finding the optimal gate by GMM for CD5
Finding the optimal gate by GMM for Fascin
Finding the optimal gate by GMM for CD45RA
Finding the optimal gate by GMM for Mac2Gal3
Finding the optimal gate by GMM for CD21
Finding the optimal gate by GMM for Vimentin
Scaling Image SeuratProject
Scaling DAPI
Scaling IRF4
Scaling CLEC9A
Scaling IRF8
Scaling TCF4
Scaling CD123
Scaling BCL-6
Scaling FOXP3
Scaling Ki67
Scaling CD25
Scaling CD40
Scaling CD10
Scaling CD11c
Scaling PAX-5
Scal

Trying to set attribute `._uns` of view, copying.


Scaling Podoplanin
Scaling HLA-DR
Scaling CD20
Scaling CD8
Scaling CD138
Scaling CD23
Scaling Fascin
Scaling CD45
Scaling CD45RA
Scaling Mac2Gal3
Scaling CD4
Scaling Granzyme B
Scaling CD21
Scaling CD3e
Scaling CD15
Scaling CD34
Scaling CD31
Scaling Vimentin
Phenotyping immune
Phenotyping Stromal
-- Subsetting immune
Phenotyping B_Cell
Phenotyping Plasma_Cell
Phenotyping T_cell
Phenotyping myeloid
-- Subsetting Plasma_Cell
Phenotyping Plasma_Cell, prolif.
-- Subsetting T_cell
Phenotyping CD8_T_Cell


/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/scimap/tools/_phenotype_cells.py:177: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/scimap/tools/_phenotype_cells.py:177: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/scimap/tools/_phenotype_cells.py:177: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice fro

Phenotyping CD4_T_Cell
Phenotyping Cytotoxic_T_Cell
Phenotyping activated_T_Cell
-- Subsetting CD4_T_Cell
Phenotyping Tfh
-- Subsetting myeloid
Phenotyping pDC
Phenotyping cDC1
Phenotyping cDC2
Phenotyping Macrophage
Phenotyping Neutrophil
Consolidating the phenotypes across all groups


/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/anndata/_core/anndata.py:1228: FutureWarning:

The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.

... storing 'new_pheno' as categorical


KeyboardInterrupt: 

In [105]:
phenotype = pd.read_csv('unknown_pheno_matrix.csv')
def gate_cell_type(d):
    print(d)
    adata = sc.read_h5ad(os.path.join(root_dir,d,'dat.h5ad'))
    # processing the data
    adata.X = adata.raw.X
    adata.X = adata.X.toarray()
    adata.raw = adata
    adata.X = np.log1p(adata.X)
    # doing the manual corrections
    adata.obs['new_pheno'] = adata.obs['cell_type_pred_knn'].astype(str)
    gate_df = pd.read_csv('cohort1_gates_orig_v2.csv', header=None, index_col=0, names=['gate'])
    adata = manual_gating(adata, gate_df)
    
    # # tracking the stromal population
    # adata.obs.loc[adata.obs['new_pheno'] == 'Stromal', 'new_pheno'] = 'Stromal, manual'
    
    # assigning the unknowns
    gate_df = pd.read_csv('cohort1_gates_orig_v2.csv', header=None)
    adata = pheno_unknown(adata, gate_df, phenotype)
    
    # reassigning B cell, germ. and FDC
    adata.obs.loc[adata.obs['new_pheno']=='FDC', 'new_pheno'] = 'Stromal'
    adata.obs.loc[adata.obs['new_pheno']=='B_Cell_germ', 'new_pheno'] = 'B_Cell'
    # accept likely-cDC1
    adata.obs.loc[adata.obs['new_pheno'] == 'likely-cDC1', 'new_pheno'] = 'cDC1'
    
    adata.obs.to_csv(os.path.join('../forQuPath/Data',d,'annotation_df_v3.csv'))
    adata.write(os.path.join(root_dir,d,'gated_dat.h5ad'))
    # return adata
    return None

In [4]:
d='MCD1_reg4'
# adata = sc.read_h5ad(os.path.join(root_dir,d,'gated_dat.h5ad'))
adata = sc.read_loom(os.path.join(root_dir,d,'gated_dat.loom'))
# adata.obs.to_csv(os.path.join('../forQuPath/Data/',d,'annotation_df_orig.csv'))
# adata.obs['new_pheno'].value_counts()
adata

/home/smithd30/miniconda3/envs/scimap/lib/python3.8/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning:

Transforming to str index.

/home/smithd30/miniconda3/envs/scimap/lib/python3.8/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning:

Transforming to str index.



AnnData object with n_obs × n_vars = 233355 × 44
    obs: 'Absolute.X', 'Absolute.Y', 'Cell.Size', 'In.Tile.X', 'In.Tile.Y', 'Reg', 'Tile.Col', 'Tile.Row', 'Tile.Z', 'Unnamed: 0', 'cell_type_pred_knn', 'cell_type_pred_knn_prob', 'index_info', 'motif', 'nCount_originalexp', 'nCount_raw', 'nFeature_originalexp', 'nFeature_raw', 'n_genes_by_counts', 'new_pheno', 'obs_names', 'orig.ident', 'total_counts'
    var: 'mean', 'mean_counts', 'n_cells_by_counts', 'pct_dropout_by_counts', 'std', 'total_counts', 'var_names'

In [101]:
adata_new = gate_cell_type(d)

MCD1_reg4
GMM for ['SeuratProject']
Categories (1, object): ['SeuratProject']
Finding the optimal gate by GMM for DAPI
Finding the optimal gate by GMM for IRF4
Finding the optimal gate by GMM for BCL-6
Finding the optimal gate by GMM for FOXP3
Finding the optimal gate by GMM for CD25
Finding the optimal gate by GMM for CD40
Finding the optimal gate by GMM for Tryptase
Finding the optimal gate by GMM for Collagen IV
Finding the optimal gate by GMM for CD134
Finding the optimal gate by GMM for CD35
Finding the optimal gate by GMM for ICOS
Finding the optimal gate by GMM for CD30
Finding the optimal gate by GMM for CD38
Finding the optimal gate by GMM for Fascin
Finding the optimal gate by GMM for CD45RA
Finding the optimal gate by GMM for Mac2Gal3
Finding the optimal gate by GMM for CD21
Finding the optimal gate by GMM for aSMA
Scaling Image SeuratProject
Scaling DAPI
Scaling IRF4
Scaling CLEC9A
Scaling IRF8
Scaling TCF4
Scaling CD123
Scaling BCL-6
Scaling FOXP3
Scaling Ki67
Scaling CD25

Trying to set attribute `._uns` of view, copying.


Scaling HLA-DR
Scaling CD20
Scaling CD8
Scaling CD138
Scaling CD23
Scaling Fascin
Scaling CD45
Scaling CD45RA
Scaling Mac2Gal3
Scaling CD4
Scaling Granzyme B
Scaling CD21
Scaling CD3e
Scaling aSMA
Scaling CD34
Scaling CD31
Scaling CD15
Phenotyping immune
Phenotyping Stromal


/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/scimap/tools/_phenotype_cells.py:177: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/scimap/tools/_phenotype_cells.py:177: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/scimap/tools/_phenotype_cells.py:177: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice fro

-- Subsetting immune
Phenotyping B_Cell
Phenotyping Plasma_Cell
Phenotyping T_cell
Phenotyping myeloid
-- Subsetting T_cell
Phenotyping CD8_T_Cell
Phenotyping CD4_T_Cell
Phenotyping Cytotoxic_T_Cell
-- Subsetting CD4_T_Cell
Phenotyping Tfh
-- Subsetting myeloid
Phenotyping pDC
Phenotyping cDC1
Phenotyping cDC2
Phenotyping Macrophage
Phenotyping Neutrophil
Consolidating the phenotypes across all groups


In [102]:
adata.obs['new_pheno'].value_counts()

CD4_T_Cell          64650
B_Cell              49455
Macrophage          33657
Endothelial         25554
CD8_T_Cell          14995
pDC                 11974
Plasma_Cell         11576
Stromal             11158
myeloid              4965
cDC1                 1916
cDC2                 1606
immune               1342
Neutrophil           1190
activated_T_Cell     1149
Lymphatics            987
Tfh                   799
Treg                  299
Cytotoxic_T_Cell      177
likely-cDC1            17
Mast_Cell               6
Name: new_pheno, dtype: int64

In [103]:
adata_new.obs['new_pheno'].value_counts()

CD4_T_Cell          64928
B_Cell              60108
Macrophage          33187
Endothelial         25554
CD8_T_Cell          15130
pDC                 11877
Plasma_Cell         11574
Stromal, manual      3814
cDC1                 1943
cDC2                 1605
myeloid              1592
Neutrophil           1107
Stromal              1011
Lymphatics            987
T_cell                906
Tfh                   834
immune                820
Treg                  299
Cytotoxic_T_Cell      190
Mast_Cell               6
Name: new_pheno, dtype: int64

In [7]:
phenotype = pd.read_csv('unknown_pheno_matrix.csv')
res = Parallel(4)(delayed(gate_cell_type)(d) for d in dat_dir)

Trying to set attribute `._uns` of view, copying.
Trying to set attribute `._uns` of view, copying.
Trying to set attribute `._uns` of view, copying.
Trying to set attribute `._uns` of view, copying.
/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/scimap/tools/_phenotype_cells.py:177: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/scimap/tools/_phenotype_cells.py:177: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

K2_reg1
GMM for ['SeuratProject']
Categories (1, object): ['SeuratProject']
Finding the optimal gate by GMM for DAPI
Finding the optimal gate by GMM for IRF4
Finding the optimal gate by GMM for BCL-6
Finding the optimal gate by GMM for FOXP3
Finding the optimal gate by GMM for CD25
Finding the optimal gate by GMM for CD40
Finding the optimal gate by GMM for CD10
Finding the optimal gate by GMM for CD134
Finding the optimal gate by GMM for CD69
Finding the optimal gate by GMM for ICOS
Finding the optimal gate by GMM for CD30
Finding the optimal gate by GMM for CD5
Finding the optimal gate by GMM for Fascin
Finding the optimal gate by GMM for CD45RA
Finding the optimal gate by GMM for Mac2Gal3
Finding the optimal gate by GMM for CD21
Finding the optimal gate by GMM for Vimentin
Scaling Image SeuratProject
Scaling DAPI
Scaling IRF4
Scaling CLEC9A
Scaling IRF8
Scaling TCF4
Scaling CD123
Scaling BCL-6
Scaling FOXP3
Scaling Ki67
Scaling CD25
Scaling CD40
Scaling CD10
Scaling CD11c
Scaling PA

Trying to set attribute `._uns` of view, copying.
Trying to set attribute `._uns` of view, copying.
Trying to set attribute `._uns` of view, copying.


K3_reg1
GMM for ['SeuratProject']
Categories (1, object): ['SeuratProject']
Finding the optimal gate by GMM for DAPI
Finding the optimal gate by GMM for IRF4
Finding the optimal gate by GMM for MX1
Finding the optimal gate by GMM for FOXP3
Finding the optimal gate by GMM for CD25
Finding the optimal gate by GMM for CD40
Finding the optimal gate by GMM for CD134
Finding the optimal gate by GMM for CD35
Finding the optimal gate by GMM for ICOS
Finding the optimal gate by GMM for CD30
Finding the optimal gate by GMM for CD5
Finding the optimal gate by GMM for Tryptase
Finding the optimal gate by GMM for Fascin
Finding the optimal gate by GMM for CD45RA
Finding the optimal gate by GMM for Mac2Gal3
Finding the optimal gate by GMM for CD21
Finding the optimal gate by GMM for Vimentin
Scaling Image SeuratProject
Scaling DAPI
Scaling IRF4
Scaling CLEC9A
Scaling IRF8
Scaling TCF4
Scaling CD123
Scaling MX1
Scaling FOXP3
Scaling Ki67
Scaling CD25
Scaling CD40
Scaling Podoplanin
Scaling CD11c
Scal

Trying to set attribute `._uns` of view, copying.
/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/scimap/tools/_phenotype_cells.py:177: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/scimap/tools/_phenotype_cells.py:177: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/scimap/tools/_phenotype_cells.py:177: SettingWithCopyWarning:


A 

...

/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/scimap/tools/_phenotype_cells.py:177: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/scimap/tools/_phenotype_cells.py:177: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/scimap/tools/_phenotype_cells.py:177: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice fro

Finding the optimal gate by GMM for Collagen IV
Finding the optimal gate by GMM for CD134
Finding the optimal gate by GMM for CD35
Finding the optimal gate by GMM for ICOS
Finding the optimal gate by GMM for CD30
Finding the optimal gate by GMM for CD38
Finding the optimal gate by GMM for Fascin
Finding the optimal gate by GMM for CD45RA
Finding the optimal gate by GMM for Mac2Gal3
Finding the optimal gate by GMM for CD21
Finding the optimal gate by GMM for aSMA
Scaling Image SeuratProject
Scaling DAPI
Scaling IRF4
Scaling CLEC9A
Scaling IRF8
Scaling TCF4
Scaling CD123
Scaling BCL-6
Scaling FOXP3
Scaling Ki67
Scaling CD25
Scaling CD40
Scaling Tryptase
Scaling CD11c
Scaling Collagen IV
Scaling PD-1
Scaling CD1c
Scaling BCL-2
Scaling CD134
Scaling CD68
Scaling CD35
Scaling CD11b
Scaling ICOS
Scaling CD163
Scaling MPO
Scaling CD30
Scaling CD38
Scaling Podoplanin
Scaling HLA-DR
Scaling CD20
Scaling CD8
Scaling CD138
Scaling CD23
Scaling Fascin
Scaling CD45
Scaling CD45RA
Scaling Mac2Gal3
S

Trying to set attribute `._uns` of view, copying.
/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/scimap/tools/_phenotype_cells.py:177: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/scimap/tools/_phenotype_cells.py:177: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/scimap/tools/_phenotype_cells.py:177: SettingWithCopyWarning:


A 

Finding the optimal gate by GMM for CD25
Finding the optimal gate by GMM for CD40
Finding the optimal gate by GMM for Tryptase
Finding the optimal gate by GMM for Collagen IV
Finding the optimal gate by GMM for CD134
Finding the optimal gate by GMM for CD35
Finding the optimal gate by GMM for ICOS
Finding the optimal gate by GMM for CD30
Finding the optimal gate by GMM for CD38
Finding the optimal gate by GMM for Fascin
Finding the optimal gate by GMM for CD45RA
Finding the optimal gate by GMM for Mac2Gal3
Finding the optimal gate by GMM for CD21
Finding the optimal gate by GMM for aSMA
Scaling Image SeuratProject
Scaling DAPI
Scaling IRF4
Scaling CLEC9A
Scaling IRF8
Scaling TCF4
Scaling CD123
Scaling BCL-6
Scaling FOXP3
Scaling Ki67
Scaling CD25
Scaling CD40
Scaling Tryptase
Scaling CD11c
Scaling Collagen IV
Scaling PD-1
Scaling CD1c
Scaling BCL-2
Scaling CD134
Scaling CD68
Scaling CD35
Scaling CD11b
Scaling ICOS
Scaling CD163
Scaling MPO
Scaling CD30
Scaling CD38
Scaling Podoplanin
S

Trying to set attribute `._uns` of view, copying.
Trying to set attribute `._uns` of view, copying.
/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/scimap/tools/_phenotype_cells.py:177: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/scimap/tools/_phenotype_cells.py:177: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/scimap/tools/_ph

In [106]:
phenotype = pd.read_csv('unknown_pheno_matrix.csv')
for d in dat_dir:
    gate_cell_type(d)

K2_reg1
GMM for ['SeuratProject']
Categories (1, object): ['SeuratProject']
Finding the optimal gate by GMM for DAPI
Finding the optimal gate by GMM for IRF4
Finding the optimal gate by GMM for BCL-6
Finding the optimal gate by GMM for FOXP3
Finding the optimal gate by GMM for CD25
Finding the optimal gate by GMM for CD40
Finding the optimal gate by GMM for CD10
Finding the optimal gate by GMM for CD134
Finding the optimal gate by GMM for CD69
Finding the optimal gate by GMM for ICOS
Finding the optimal gate by GMM for CD30
Finding the optimal gate by GMM for CD5
Finding the optimal gate by GMM for Fascin
Finding the optimal gate by GMM for CD45RA
Finding the optimal gate by GMM for Mac2Gal3
Finding the optimal gate by GMM for CD21
Finding the optimal gate by GMM for Vimentin
Scaling Image SeuratProject
Scaling DAPI
Scaling IRF4
Scaling CLEC9A
Scaling IRF8
Scaling TCF4
Scaling CD123
Scaling BCL-6
Scaling FOXP3
Scaling Ki67
Scaling CD25
Scaling CD40
Scaling CD10
Scaling CD11c
Scaling PA

Trying to set attribute `._uns` of view, copying.


Scaling CD34
Scaling CD31
Scaling Vimentin
Phenotyping immune
Phenotyping Stromal
-- Subsetting immune
Phenotyping B_Cell
Phenotyping Plasma_Cell
Phenotyping T_cell
Phenotyping myeloid
-- Subsetting T_cell
Phenotyping CD8_T_Cell
Phenotyping CD4_T_Cell
Phenotyping Cytotoxic_T_Cell
-- Subsetting CD4_T_Cell
Phenotyping Tfh
-- Subsetting myeloid
Phenotyping pDC
Phenotyping cDC1
Phenotyping cDC2
Phenotyping Macrophage
Phenotyping Neutrophil
Consolidating the phenotypes across all groups


/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/scimap/tools/_phenotype_cells.py:177: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/scimap/tools/_phenotype_cells.py:177: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/scimap/tools/_phenotype_cells.py:177: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice fro

MCD1_reg1
GMM for ['SeuratProject']
Categories (1, object): ['SeuratProject']
Finding the optimal gate by GMM for DAPI
Finding the optimal gate by GMM for IRF4
Finding the optimal gate by GMM for BCL-6
Finding the optimal gate by GMM for FOXP3
Finding the optimal gate by GMM for CD25
Finding the optimal gate by GMM for CD40
Finding the optimal gate by GMM for Tryptase
Finding the optimal gate by GMM for Collagen IV
Finding the optimal gate by GMM for CD134
Finding the optimal gate by GMM for CD35
Finding the optimal gate by GMM for ICOS
Finding the optimal gate by GMM for CD30
Finding the optimal gate by GMM for CD38
Finding the optimal gate by GMM for Fascin
Finding the optimal gate by GMM for CD45RA
Finding the optimal gate by GMM for Mac2Gal3
Finding the optimal gate by GMM for CD21
Finding the optimal gate by GMM for aSMA
Scaling Image SeuratProject
Scaling DAPI
Scaling IRF4
Scaling CLEC9A
Scaling IRF8
Scaling TCF4
Scaling CD123
Scaling BCL-6
Scaling FOXP3
Scaling Ki67
Scaling CD25

...

Trying to set attribute `._uns` of view, copying.


Scaling Vimentin
Phenotyping immune
Phenotyping Stromal
-- Subsetting immune
Phenotyping B_Cell
Phenotyping Plasma_Cell
Phenotyping T_cell
Phenotyping myeloid
-- Subsetting T_cell
Phenotyping CD8_T_Cell
Phenotyping CD4_T_Cell
Phenotyping Cytotoxic_T_Cell
-- Subsetting CD4_T_Cell
Phenotyping Tfh


/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/scimap/tools/_phenotype_cells.py:177: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/scimap/tools/_phenotype_cells.py:177: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/scimap/tools/_phenotype_cells.py:177: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice fro

-- Subsetting myeloid
Phenotyping pDC
Phenotyping cDC1
Phenotyping cDC2
Phenotyping Macrophage
Phenotyping Neutrophil
Consolidating the phenotypes across all groups


/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/anndata/_core/anndata.py:1228: FutureWarning:

The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.

... storing 'new_pheno' as categorical


In [12]:
adata = sc.read_h5ad('../../outs/neighborhood_analysis/HVCD2_reg2/dat.h5ad')
adata.obs['new_pheno'].value_counts()
# adata.obs['cell_type_pred_knn'].value_counts()


CD4_T_Cell          55605
B_Cell              43769
Macrophage          17312
Endothelial         17149
CD8_T_Cell          15955
FDC                 11237
pDC                  5979
cDC1                 2816
Sinus_histiocyte     1741
cDC2                 1105
Tfh                   970
B_Cell_germ           691
activated_T_Cell      504
Plasma_Cell           495
Treg                  371
unknown               274
Plasma_prolif         255
Lymphatics            219
Neutrophil            151
Cytotoxic_T_Cell      144
Macrophage_M2          15
Name: cell_type_pred_knn, dtype: int64

In [11]:
adata

AnnData object with n_obs × n_vars = 176757 × 44
    obs: 'Unnamed: 0', 'orig.ident', 'nCount_originalexp', 'nFeature_originalexp', 'Reg', 'Tile.Row', 'Tile.Col', 'Tile.Z', 'In.Tile.Y', 'In.Tile.X', 'Absolute.Y', 'Absolute.X', 'Cell.Size', 'n_genes_by_counts', 'total_counts', 'nCount_raw', 'nFeature_raw', 'cell_type_pred_knn', 'cell_type_pred_knn_prob', 'motif', 'index_info'
    var: 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'mean', 'std'
    uns: 'spatial_distance', 'spatial_interaction', 'spatial_lda', 'spatial_lda_probability'